In [1]:
import numpy as np
from random import random
import helpers as hlp
from tqdm import tqdm

Want to sample from three variables:
1. $\lambda$
2. $\tau$
3. $\beta$

-> these values are then averaged to get the Monte-Carlo estimator
-> first: what is density of $\beta$?

In [2]:
# import data from DNN training
extracted_coefficients_directory = '../../../data/commaai/extracted_coefficients/20201027_filtered_gaussian_resampled/'
B_zeta_path = str(extracted_coefficients_directory + 'Bzeta/B_zeta.npy')
beta_path = str(extracted_coefficients_directory + 'beta/beta.csv')
z_path = str(extracted_coefficients_directory + 'Bzeta/tr_labels.npy')
beta = np.genfromtxt(beta_path, delimiter=',')
B_zeta = np.load(B_zeta_path)
B_zeta = B_zeta.reshape(B_zeta.shape[0], beta.shape[0])
z = np.load(z_path)

In [ ]:
beta = np.repeat(0,10)
# number of samples we want to produce
M = 10
L = 2

# number of parameters of theta
q = B_zeta.shape[1]*2 + 1
p = B_zeta.shape[1]

n = B_zeta.shape[0]

# start values for beta, lambda, tau
# beta is taken from trained network and other two
# are initialized randomly
theta_m_1 = np.append(np.append(beta, np.random.rand(B_zeta.shape[1],)), np.random.rand(1,))

# stepsize
epsilon = 0.1

r_m = np.zeros(theta_m_1.shape[0])

 need to write functions for
- log-density of $\log \lambda_j^2$
- gradient of log-density of $\log \lambda_j^2$

- log-density of $\log \tau$
- gradient of $\log \tau$

- log-density of $\beta$
- gradient of $\beta$

In [4]:
# log density of tau
def log_density_tau(tau, p, Lambda_vec):
    return(-(p-1)*log(tau) - log(1 + tau**2) - np.sum(Lambda**2)/(tau**2))

In [5]:
# log density of lambda_j
def log_density_lambda_j(const, lambda_j, beta_j, S, z, B, beta_t):
    const = -(1/2)*np.sum(np.log(S**2)) - (1/2)*z.dot(np.linalg.solve(S.dot(S))).dot(z)  + beta_t.dot(B.T).dot(S**(-1)).dot(z)
    return( const - (1/2)*(log(lambda_j**2)  + (beta_j**2)/(lambda_j**2) + 2*log(1 + (lambda_j**2)/(tau**2) + (1/2)*lambda_j)))
# last term might be wrong bc doesnt exist in kleins paper...


In [6]:
theta_m_1

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.69889898, 0.65634294, 0.06012714, 0.08063367, 0.37048848,
       0.4406087 , 0.93624851, 0.19854791, 0.8774273 , 0.44997246,
       0.19465204])

In [7]:
def Leapfrog(theta, r, epsilon, n, z, p):
    
    # compute gradient with theta
    Delta_theta = hlp.Delta_theta(theta, B_zeta, n, z, p)
    
    # update momentum
    r_tilde = r + (epsilon/2)*Delta_theta
    
    # update theta
    theta_tilde = theta + epsilon*r_tilde
    
    # compute updated gradient
    Delta_theta_tilde = hlp.Delta_theta(theta_tilde, B_zeta, n, z, p)
    
    # update momentum again
    r_tilde = r_tilde + (epsilon/2)*Delta_theta_tilde
    
    return(theta_tilde, r_tilde)

In [ ]:
r0 = np.repeat(None, M)
theta_tilde = np.repeat(None, M)
r_tilde = np.repeat(None, M)
log_dens =  np.repeat(None, M)
alpha = np.repeat(None, M)
theta_m_1 = np.repeat(None, M)
r_m = np.repeat(None, M)
theta_m_1[0] = np.append(np.append(beta, np.random.rand(B_zeta.shape[1],)), np.random.rand(1,))

acc = []
# loop over number of samples that we want to produce
for m in tqdm(range(0, M)):
    print(theta_m_1[0].shape)
    
    # Update S
    # draw momentum from normal distribution
    r0[m] = np.random.multivariate_normal(np.zeros(q), np.identity(q), 1)
    
    # set new parameters
    theta_tilde[m] = theta_m_1[m].reshape(21,)
    r_tilde[m] = r0[m]
    
    # generate proposal through L leapfrog updates 
    for i in range(0,L):
        theta_tilde[m], r_tilde[m] = Leapfrog(theta_tilde[m], r_tilde[m], epsilon, n, z, p)
    
    # probability that proposal is accepted
    log_dens[m] = log_density(theta_tilde[m])
    alpha[m] = min([1, (np.exp(log_dens[m] - r_tilde[m]*r_tilde[m]*1/2))/(log_dens[m] - r0[m]*r0[m]*1/2)])
    
    decision = rand()
    if rand <= alpha:
        theta_m_1[m + 1] = theta_tilde[m]
        r_m[m + 1] = - r_tilde[m]
        acc.append(1)
    else:
        theta_m_1[m + 1] = theta_tilde[m - 1]
        r_m[m + 1] = - r_tilde[m - 1]
        acc.append(0)
    

In [ ]:
theta_m_1.shape